### Import the library

In [35]:
# Import the library
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from datetime import datetime, timedelta
import pickle

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings('ignore')

### Load the dataset

In [2]:
# Load the dataset from parquet file
df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet')
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1.0,1.72,1.0,N,186,79,2,17.7,1.0,0.5,0.00,0.0,1.0,22.70,2.5,0.0
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1.0,1.80,1.0,N,140,236,1,10.0,3.5,0.5,3.75,0.0,1.0,18.75,2.5,0.0
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1.0,4.70,1.0,N,236,79,1,23.3,3.5,0.5,3.00,0.0,1.0,31.30,2.5,0.0
3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,1.0,1.40,1.0,N,79,211,1,10.0,3.5,0.5,2.00,0.0,1.0,17.00,2.5,0.0
4,1,2024-01-01 00:46:51,2024-01-01 00:52:57,1.0,0.80,1.0,N,211,148,1,7.9,3.5,0.5,3.20,0.0,1.0,16.10,2.5,0.0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2964624 entries, 0 to 2964623
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int32         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int32         
 8   DOLocationID           int32         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  Airport_fee           

### Exploratory Data Analysis (EDA)

In [4]:
# 1. Distribution of number passenger per trip
df['passenger_count'].value_counts()

passenger_count
1.0    2188739
2.0     405103
3.0      91262
4.0      51974
5.0      33506
0.0      31465
6.0      22353
8.0         51
7.0          8
9.0          1
Name: count, dtype: int64

In [5]:
# 2. Average miles for trip distance
# Next action : check for the trip distance anomaly
np.round(df['trip_distance'].describe(), 2)

count    2964624.00
mean           3.65
std          225.46
min            0.00
25%            1.00
50%            1.68
75%            3.11
max       312722.30
Name: trip_distance, dtype: float64

In [6]:
# 3. How is the type of rate?
# Next action : what is type 99
df['RatecodeID'].value_counts()

RatecodeID
1.0     2663350
2.0       98713
99.0      28663
5.0       19410
3.0        7954
4.0        6365
6.0           7
Name: count, dtype: int64

In [7]:
# 4. How is the signal connection on the taxi trip?
# Next action : find possibility areas having signal problem
df['store_and_fwd_flag'].value_counts()

store_and_fwd_flag
N    2813126
Y      11336
Name: count, dtype: int64

In [8]:
# 5. Most pick up and drop off area
# Next action : find pair of pick up and drop off area most likely ordered

# Pick up area
df['PULocationID'].value_counts()

PULocationID
132    145240
161    143471
237    142708
236    136465
162    106717
        ...  
105         1
44          1
111         1
204         1
109         1
Name: count, Length: 260, dtype: int64

In [9]:
# Drop off area
df['DOLocationID'].value_counts()

DOLocationID
236    142044
237    130249
161    111942
230     90603
142     89673
        ...  
5           9
44          5
2           4
99          1
105         1
Name: count, Length: 261, dtype: int64

In [10]:
# 6. How the customer tend to pay the trip
# Next action : check on tip_amount field for cash payment type
df['payment_type'].value_counts()

payment_type
1    2319046
2     439191
0     140162
4      46628
3      19597
Name: count, dtype: int64

In [11]:
# 7. How is the distribution of base price
# Next action : what is fare_amount negative meaning?
np.round(df['fare_amount'].describe(), 2)

count    2964624.00
mean          18.18
std           18.95
min         -899.00
25%            8.60
50%           12.80
75%           20.50
max         5000.00
Name: fare_amount, dtype: float64

In [12]:
# 8. How is the distribution of extra price
# Next action : what is extra negative meaning, how is the mechanism of charge. Do they apply prorate?
np.round(df['extra'].describe(), 2)

count    2964624.00
mean           1.45
std            1.80
min           -7.50
25%            0.00
50%            1.00
75%            2.50
max           14.25
Name: extra, dtype: float64

In [13]:
# 9. How is the distribution of mta_tax
# Next action : there are value except (0.5 and -0.5). What is that mean?
np.round(df['mta_tax'].value_counts(), 2)

mta_tax
 0.5    2900474
-0.5      34434
 0.0      29707
 4.0          5
 1.6          1
 0.8          1
 1.4          1
 3.0          1
Name: count, dtype: int64

In [14]:
# 10. How is the distribution of improvement surcharge
# Next action : the distribution of improvement surcharge is similar to each other
np.round(df['improvement_surcharge'].value_counts(), 2)

improvement_surcharge
 1.0    2927710
-1.0      35500
 0.0        838
 0.3        574
-0.3          2
Name: count, dtype: int64

In [15]:
# 11. How is the distribution of congestion surcharge
# Next action : why there is negative charge
df['congestion_surcharge'].value_counts()

congestion_surcharge
 2.50    2577755
 0.00     217877
-2.50      28824
 0.75          3
 1.00          2
-0.75          1
Name: count, dtype: int64

In [16]:
# 12. How is the distribution of airport fee
# Next action : why there is negative value
df['Airport_fee'].value_counts()

Airport_fee
 0.00    2586789
 1.75     232752
-1.75       4921
Name: count, dtype: int64

In [17]:
# 13. How is the distribution of total_amount
# Next action : why there is negative value
np.round(df['total_amount'].describe(), 2)

count    2964624.00
mean          26.80
std           23.39
min         -900.00
25%           15.38
50%           20.10
75%           28.56
max         5000.00
Name: total_amount, dtype: float64

In [19]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1.0,1.72,1.0,N,186,79,2,17.7,1.0,0.5,0.00,0.0,1.0,22.70,2.5,0.0
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1.0,1.80,1.0,N,140,236,1,10.0,3.5,0.5,3.75,0.0,1.0,18.75,2.5,0.0
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1.0,4.70,1.0,N,236,79,1,23.3,3.5,0.5,3.00,0.0,1.0,31.30,2.5,0.0
3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,1.0,1.40,1.0,N,79,211,1,10.0,3.5,0.5,2.00,0.0,1.0,17.00,2.5,0.0
4,1,2024-01-01 00:46:51,2024-01-01 00:52:57,1.0,0.80,1.0,N,211,148,1,7.9,3.5,0.5,3.20,0.0,1.0,16.10,2.5,0.0


### Data Preparation

In [58]:
def load_dataset(file_name):
    # Condition for type of file
    if file_name.endswith('.csv'):
        # Load the csv dataset
        df = pd.read_csv(file_name)
        
        # Convert column type into datetime
        df.loc[:, 'tpep_dropoff_datetime'] = pd.to_datetime(df.loc[:, 'tpep_dropoff_datetime'])
        df.loc[:, 'tpep_pickup_datetime'] = pd.to_datetime(df.loc[:, 'tpep_pickup_datetime'])
        
    elif file_name.endswith('.parquet'):
        # Load the parquet dataset
        df = pd.read_parquet(file_name)
    
    # The duration of trip in minutes
    df.loc[:, 'duration'] = df.loc[:, 'tpep_dropoff_datetime'] - df.loc[:, 'tpep_pickup_datetime']
    df.loc[:, 'duration'] = df.loc[:, 'duration'].apply(lambda td : td.total_seconds() / 60)
    
    # Convert column type into string
    categorical = ['PULocationID', 'DOLocationID']
    df.loc[:, categorical] = df.loc[:, categorical].astype('str')
    
    return df

def preprocess_dataset(df):
    # Filter the duration of trip between 1 - 60 minutes
    df = df.loc[(df['duration'] >= 1) & (df['duration'] <= 60), :]
    
    # Concatenate pair pick-up and drop-off location
    df.loc[:, 'PU_DO'] = df.loc[:, 'PULocationID'] + '_' + df.loc[:, 'DOLocationID']
    
    return df

In [59]:
# Define the location of files
file_name_train = 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet'
file_name_test = 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-02.parquet'

# Load the dataset using custom function
df_train = load_dataset(file_name_train)
df_val = load_dataset(file_name_test)

# Preprocess the dataset
df_train_filtered = preprocess_dataset(df_train)
df_val_filtered = preprocess_dataset(df_val)

In [61]:
# Define the type of variable we want to use
categorical = ['PU_DO'] # ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']
target = 'duration'

Data Train

In [62]:
# Data preparation for training data
train_dicts = df_train_filtered[categorical + numerical].to_dict(orient='records')

# Vectorization for feature and target
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)
y_train = df_train_filtered[target].values

# Modelling
lr = LinearRegression()
lr.fit(X_train, y_train)

# Evaluation
y_pred = lr.predict(X_train)
mean_squared_error(y_train, y_pred, squared=False)

5.297035732120616

Data Validation

In [63]:
# Data prepartion fo validation data
val_dicts = df_val_filtered[categorical + numerical].to_dict(orient='records')

# Transform the vectorizer from train pattern
X_val = dv.transform(val_dicts)
y_val = df_val_filtered.loc[:, target].values

# Predict the validation data based on model build using training data 
y_pred = lr.predict(X_val)
mean_squared_error(y_val, y_pred, squared=False)

5.5059366623994395